In [18]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
from sklearn.metrics import r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from xgboost import XGBRegressor

In [2]:
df= pd.read_csv("data/최종_최종__월별기상관측.csv",encoding="cp949")
df= df.drop(labels="Unnamed: 0", axis=1)
df

,행정구역,지점명,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2)
0,강원도,속초,2000-01,-0.4,9.9,-10.9,90.2,152.990
1,강원도,속초,2000-02,-0.1,9.2,-8.6,8.4,329.070
2,강원도,속초,2000-03,6.9,18.5,-4.2,8.7,448.650
3,강원도,속초,2000-04,11.2,24.1,2.4,23.4,486.355
4,강원도,속초,2000-05,14.3,30.5,6.2,71.0,520.280
...,...,...,...,...,...,...,...,...
23305,경상남도,남해,2021-08,26.1,35.5,20.7,473.2,481.070
23306,경상남도,남해,2021-09,22.2,30.1,16.3,164.6,366.930
23307,경상남도,남해,2021-10,17.2,28.9,4.7,43.6,429.240
23308,경상남도,남해,2021-11,10.7,21.7,1.4,53.5,323.960


In [3]:
output_raw= pd.read_csv('data/배추_생산량.csv')

In [4]:
grouped= df.groupby(['행정구역', '일시']).mean()

In [5]:
grouped= grouped.reset_index()
grouped['면적당 생산량']= 0
grouped

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),면적당 생산량
0,강원도,2000-01,-2.681818,8.690909,-14.745455,58.272727,158.358182,0
1,강원도,2000-02,-2.872727,7.827273,-14.318182,6.509091,322.926364,0
2,강원도,2000-03,4.790909,19.000000,-7.936364,13.790909,433.992727,0
3,강원도,2000-04,10.154545,23.518182,-1.872727,37.354545,481.298636,0
4,강원도,2000-05,15.618182,31.500000,4.181818,75.181818,518.235455,0
...,...,...,...,...,...,...,...,...
4483,충청북도,2021-08,24.160000,34.240000,17.360000,241.860000,445.145000,0
4484,충청북도,2021-09,20.680000,29.100000,12.920000,191.580000,426.565000,0
4485,충청북도,2021-10,13.760000,29.460000,-1.000000,35.260000,407.200000,0
4486,충청북도,2021-11,6.800000,19.880000,-4.900000,47.800000,243.284000,0


In [6]:
#봄:3,4,5/여름:6,7,8/가을:9,10,11/겨울:12,1,2
city_list= ['강원도', '경기도', '인천광역시', '서울특별시', '경상북도', '충청북도', '충청남도', '대전광역시','전라남도', '대구광역시', '전라북도', '울산광역시', '경상남도', '광주광역시', '부산광역시', '제주도','세종특별자치시']
yr_list= np.arange(2000, 2022)
detail= ['.1', '.4', '.7', '.10']
month= ["-01", "-02", "-03", "-04", "-05", "-06", "-07", "-08", "-09", "-10", "-11", "-12"]

for i in range(len(city_list)):
    for yr in yr_list:
        try:
            for l in range(len(detail)):
                val= str(yr) + detail[l]
                if l==0:
                    for k in range(2,5):
                        dt= str(yr) + month[k]
                        ind= grouped[grouped['행정구역']==city_list[i]][grouped['일시']==dt].index
                        grouped.loc[ind, '면적당 생산량']= output_raw.loc[city_list[i], val]  
                elif l==1:
                    for k in range(5,8):
                        dt= str(yr) + month[k]
                        ind= grouped[grouped['행정구역']==city_list[i]][grouped['일시']==dt].index
                        grouped.loc[ind, '면적당 생산량']= output_raw.loc[city_list[i], val]
                elif l==2:
                    for k in range(8,11):
                        dt= str(yr) + month[k]
                        ind= grouped[grouped['행정구역']==city_list[i]][grouped['일시']==dt].index
                        grouped.loc[ind, '면적당 생산량']= output_raw.loc[city_list[i], val]
                else:
                    num_list=[11,0,1]
                    for k in range(len(num_list)):
                        dt= str(yr) + month[num_list[k]]
                        ind= grouped[grouped['행정구역']==city_list[i]][grouped['일시']==dt].index
                        grouped.loc[ind, '면적당 생산량']= output_raw.loc[city_list[i], val]  
        except:
            pass

C:\Users\jhahn\AppData\Local\Temp\ipykernel_6004\145606724.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ind= grouped[grouped['행정구역']==city_list[i]][grouped['일시']==dt].index


In [7]:
grouped

,행정구역,일시,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),면적당 생산량
0,강원도,2000-01,-2.681818,8.690909,-14.745455,58.272727,158.358182,0
1,강원도,2000-02,-2.872727,7.827273,-14.318182,6.509091,322.926364,0
2,강원도,2000-03,4.790909,19.000000,-7.936364,13.790909,433.992727,0
3,강원도,2000-04,10.154545,23.518182,-1.872727,37.354545,481.298636,0
4,강원도,2000-05,15.618182,31.500000,4.181818,75.181818,518.235455,0
...,...,...,...,...,...,...,...,...
4483,충청북도,2021-08,24.160000,34.240000,17.360000,241.860000,445.145000,0
4484,충청북도,2021-09,20.680000,29.100000,12.920000,191.580000,426.565000,0
4485,충청북도,2021-10,13.760000,29.460000,-1.000000,35.260000,407.200000,0
4486,충청북도,2021-11,6.800000,19.880000,-4.900000,47.800000,243.284000,0


In [8]:
train= grouped[grouped['일시'].str.contains('2021') == False]
test= grouped[grouped['일시'].str.contains('2021')]

grouped= grouped.drop(labels='일시', axis=1)

In [29]:
grouped[grouped.iloc[:,-1] == 0]

,행정구역,평균기온(°C),최고기온(°C),최저기온(°C),월합강수량(00~24h만)(mm),합계 일사량(MJ/m2),면적당 생산량
0,강원도,-2.681818,8.690909,-14.745455,58.272727,158.358182,0
1,강원도,-2.872727,7.827273,-14.318182,6.509091,322.926364,0
2,강원도,4.790909,19.000000,-7.936364,13.790909,433.992727,0
3,강원도,10.154545,23.518182,-1.872727,37.354545,481.298636,0
4,강원도,15.618182,31.500000,4.181818,75.181818,518.235455,0
...,...,...,...,...,...,...,...
4483,충청북도,24.160000,34.240000,17.360000,241.860000,445.145000,0
4484,충청북도,20.680000,29.100000,12.920000,191.580000,426.565000,0
4485,충청북도,13.760000,29.460000,-1.000000,35.260000,407.200000,0
4486,충청북도,6.800000,19.880000,-4.900000,47.800000,243.284000,0


In [10]:
# window_size = 3

# data_X = []
# data_y = []
# for i in range(len(y_sc) - window_size):
#     _X = X_sc[i : i + window_size] # 다음 날 종가(i+windows_size)는 포함되지 않음
#     _y = y_sc[i + window_size]     # 다음 날 종가
#     data_X.append(_X)
#     data_y.append(_y)
# print(_X, "->", _y)

In [14]:
X = grouped.iloc[:,1:6]
y = grouped.iloc[:,-1]

mmsc_X = MinMaxScaler()
mmsc_y = MinMaxScaler()

mmsc_X.fit(X)
mmsc_y.fit(y.values.reshape(-1,1))

X_sc = mmsc_X.transform(X)
y_sc = mmsc_y.transform(y.values.reshape(-1,1))

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_sc, y_sc, test_size=0.2, random_state=9)

In [20]:
eta = np.logspace(-3,2,6)   # xgboost 해당
n_estimators = [15,20,50,100,200,500]   # randomforesteregressor(사용할 tree의 개수), gradientboost, adaboost, xgboost 해당
max_depth = [2,3,5,7,10,12]  # randomforesteregressor(tree의 깊이), gradientboost, xgboost 해당
gamma = [0, 0.05, 0.1, 0.2, 0.5]    # xgboost 해당
max_leaf_nodes = [2,3,5,7]     # randomforest, xgboost 해당
min_child_weight = [1,2,3,5]   # xgboost 해당

params_xgb = {"learning_rate" : eta,
                "n_estimators" : n_estimators, 
                "max_depth" : max_depth, 
                "gamma" : gamma,
                "min_child_weight" : min_child_weight}

In [22]:
kfold = KFold(n_splits=5, shuffle=True, random_state=9)

xgb = XGBRegressor()

grid_xgb = GridSearchCV(estimator=xgb, param_grid=params_xgb, cv=kfold, n_jobs=-1, scoring="neg_mean_squared_error")

In [23]:
grid_xgb.fit(X_train, y_train)

c:\Users\jhahn\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:953: UserWarning: One or more of the test scores are non-finite: [-0.2426103  -0.2401959  -0.22620597 ...         nan         nan
         nan]
  warnings.warn(


GridSearchCV(cv=KFold(n_splits=5, random_state=9, shuffle=True),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None, gpu_id=None,
                                    grow_policy=None, importance_type=None,
                                    inter...
                                    monotone_constraints=None, n_estimators=100,
                                    n_jobs=None, num_parallel_tree=None,
                                    predictor=None, random_state=None, ...),
             n_jobs=-1,
             param_grid={'gamma': [0, 0.05, 0.1, 0.2, 0.5],
                         'learning_rate': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02]),
                         'max_depth': [2, 3, 5, 7, 10, 12],
                         'min_child_weight': [1, 2, 3, 5],
                         'n_estimators': [15, 20, 50, 100, 200, 500]},
             scoring='neg_mean_squared_error')

In [24]:
print("best parameter :", grid_xgb.best_params_)
print("best score :", grid_xgb.best_score_)

best parameter : {'gamma': 0, 'learning_rate': 1.0, 'max_depth': 2, 'min_child_weight': 1, 'n_estimators': 15}
best score : 0.0


In [27]:
em = grid_xgb.best_estimator_
y_pred = em.predict(X_test)
print("r2_score :",r2_score(y_test, y_pred))

r2_score : 1.0
